<a href="https://colab.research.google.com/github/UITTrinhQuangTruong/CS114.K21/blob/master/Do_An_LinhTinh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Tiền xử lý dữ liệu và trích xuất đặc trưng 

**hàm xử lý dữ liệu và trích xuất đặc trưng bằng HOG**

In [2]:
def CvCanny(matrix):
  v = np.median(matrix)
  sigma = 0.33
  lower = int(max(0, (1.0 - sigma) * v))
  upper = int(min(255, (1.0 + sigma) * v))
  canny = cv2.Canny(matrix, lower, upper, True)
  return canny

In [3]:
def apply_roi(img, roi):
  #thresh, roi = cv2.threshold(roi, thresh=128, maxval=1, type=cv2.THRESH_BINARY)
  new_img = img * roi
  return new_img

In [4]:
def extract_features(path, label, data, labels):
  videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
  for videoPath in videoPaths:
    cap = cv2.VideoCapture(videoPath)
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (160, 120))
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    s = 1
    roi = []
    while(cap.isOpened()):
        ret, frame2 = cap.read()

        if not ret:
          break;
        if s % 2 == 0:
          frame2 = cv2.resize(frame2, (160, 120))
          #Trích xuất dòng quang từ 2 frame gần nhau
          next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
          optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
          flow = optical_flow.calc(prvs, next, None)
          if s == 3:
            flowx = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
            flowy = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
          mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
          hsv[...,0] = ang*180/np.pi/2
          nom = np.zeros((160,120))
          hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
          bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
          gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

          #Tăng độ nét cho ảnh dòng quang
          '''kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
          sharpen = cv2.filter2D(hsv, -1, kernel)
          canny1 = CvCanny(sharpen)'''
          canny2 = CvCanny(gray)
          #gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
          #cv2_imshow(gray)
          
          '''gray1 = cv2.filter2D(gray, -1, kernel)[...,::-1]
          gray = cv2.ximgproc.guidedFilter(gray, gray1, radius=2, eps=0.004, dDepth=-1)[...,::-1]
          canny3 = CvCanny(gray)
          cv2_imshow(gray)'''
          

          kernel = np.ones((5,5),np.uint8)
          closing = cv2.morphologyEx(canny2, cv2.MORPH_CLOSE, kernel)
          closing_inv = cv2.bitwise_not(closing)
          
          h, w = closing.shape[:2]
          mask = np.zeros((h+2, w+2), np.uint8)
          
          '''im_floodfill = closing.copy()
          cv2.floodFill(im_floodfill, mask, (0,0), 255);
          im_floodfill_inv = cv2.bitwise_not(im_floodfill)
          floodfill = closing | im_floodfill'''

          contour,hier = cv2.findContours(closing,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

          for cnt in contour:
            cv2.drawContours(closing,[cnt],0,255,-1)

          opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
          #opening = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
          '''cv2_imshow(canny1)
          cv2_imshow(canny2)
          cv2_imshow(canny3)'''

          after_roi = apply_roi(next, opening)

          # trích xuất feature bằng HOG
          winSize = (32,32)
          blockSize = (16,16)
          blockStride = (8,8)
          cellSize = (8,8)
          nbins = 9
          hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
          image_hog = hog.compute(after_roi)

          data.append(image_hog)
          labels.append(label)


          prvs = next.copy()


        s += 1
    print(videoPath + "---done--")



**Khởi tạo các biến chứa nhãn và dữ liệu**

In [5]:
labels = []
data = []

**Tiến hành trích xuất đặc trưng lưu vào các biến data và labels**

In [6]:
path = "/content/drive/My Drive/Data/cobaoluc"
extract_features(path, 0, data, labels)
path = "/content/drive/My Drive/Data/khongbaoluc"
extract_features(path, 1, data, labels)



/content/drive/My Drive/Data/cobaoluc/0.mp4---done--
/content/drive/My Drive/Data/cobaoluc/1.mp4---done--
/content/drive/My Drive/Data/cobaoluc/45.mp4---done--
/content/drive/My Drive/Data/cobaoluc/75.mp4---done--
/content/drive/My Drive/Data/cobaoluc/95.mp4---done--
/content/drive/My Drive/Data/cobaoluc/98.mp4---done--
/content/drive/My Drive/Data/cobaoluc/7.mp4---done--
/content/drive/My Drive/Data/cobaoluc/68.mp4---done--
/content/drive/My Drive/Data/cobaoluc/71.mp4---done--
/content/drive/My Drive/Data/cobaoluc/88.mp4---done--


KeyboardInterrupt: ignored

In [ ]:
print(len(data))
print(len(labels))

**Xử lý các feature để có thêm đưa vào model train được**

In [ ]:
data = np.transpose(data)[0]

**Chia dữ liệu train và test**

In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(data, labels,
	test_size=0.20, random_state=42)

# Train model

**Model svm**

In [ ]:
from sklearn import svm

lsvc = svm.SVC()
lsvc.fit(trainX, trainY)

print(lsvc.score(trainX, trainY))
print(lsvc.score(testX, testY))
  

0.9558823529411765
0.6379310344827587


**Model RandomForestClassifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators= 10, random_state= 0)

rfc.fit(trainX, trainY)

print(rfc.score(trainX, trainY))
print(rfc.score(testX, testY))

**Model GaussianNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()


gnb.fit(trainX, trainY)

print(gnb.score(trainX, trainY))
print(gnbfc.score(testX, testY))


**Model LogisticRegression**

In [ ]:

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(trainX, trainY)

print(lr.score(trainX, trainY))
print(lr.score(testX, testY))
